# 22/02/2021

In [1]:
import ROOT as r
import numpy as np

Welcome to JupyROOT 6.22/02


In [2]:
# Now we create the file and put both trees into it
file = r.TFile("stationaryW_case.root","recreate")

I am creating two trees. 

One for the momenta components of electron called `e` and one for neutrino called `n`.

Each tree will have 4 branches for `Px`,`Py`,`Pz` and `E`.

In [3]:
e = r.TTree("e", "electronTree")
n = r.TTree("n", "neutrinoTree")

Now I will use the `TLorentzVector` function to generate the data. In this case it is not necessary I suppose, but afterwards, once we need to use `boost` or other things, it will be better if we start with `TLorentzVector` now already.

In [4]:
# Function to generate initial random momentum components for electron
def elMomComponents(mag):
    ''' chooses random theta and phi and then uses it to calculate 
        initial random components of electron momentum
        
        Input:
        mag    magnitude of the momentum
        
        Returns:
        Px
        Py
        Pz
    '''
    theta = np.random.uniform(0,np.pi)
    phi = np.random.uniform(0,2*np.pi)
    
    Px = mag * np.sin(theta) * np.cos(phi) 
    Py = mag * np.sin(theta) * np.sin(phi)
    Pz = mag * np.cos(theta)
    
    return Px, Py, Pz

In [5]:
# Setting up 4Vectors for electron and neutrino
e4Vect = r.TLorentzVector() #electron
n4Vect = r.TLorentzVector() #neutrino

In [6]:
# Invariant mass of W boson
massW = 80.3 # Gev/c^2

# energy split equaly in the decay -> massW/2
productE = massW/2

In [7]:
# Setting up the variables, so we can set up branches, where we store them
ePx = np.array([e4Vect.Px()])
ePy = np.array([e4Vect.Py()])
ePz = np.array([e4Vect.Pz()])
eE = np.array([productE])

In [8]:
# Setting up branches to store components of electron momenta 
# (each component is placed separately on each branch)
e.Branch("Px",ePx,"xComponent[1]/D")
e.Branch("Py",ePy,"yComponent[1]/D")
e.Branch("Pz",ePz,"zComponent[1]/D")
e.Branch("E",eE,"energy[1]/D")

In [9]:
# Function to generate 4vector and splits it into different branches
def elMomFill(j, mag, m):
    ''' Uses function elMomComponents to generate random values
        of momenta components with given magnitude of the vector.
        Then it uses TLorentzVector to fill the branch of electron momenta components
        with these randomly generated values.
        
        Input:
        j     number of generated momenta
        mag   magnitude of generated momenta 
        m     energy of the particle
    '''
    
    for i in range(j):
        Px, Py, Pz = elMomComponents(mag)
        e4Vect.SetPxPyPzE(Px, Py, Pz, m)

        # Mass won't change, so we do not need it here
        ePx[0] = e4Vect.Px()
        ePy[0] = e4Vect.Py()
        ePz[0] = e4Vect.Pz()
        eE[0] = e4Vect.E()

        e.Fill()

In [10]:
elMomFill(3,1,productE)

In [11]:
e.Scan("Px:Py:Pz:E")

3

************************************************************
*    Row   *        Px *        Py *        Pz *         E *
************************************************************
*        0 * -0.932665 * -0.050281 * -0.357221 *     40.15 *
*        1 * 0.4708793 * -0.563725 * -0.678591 *     40.15 *
*        2 * 0.4350746 * -0.303448 * 0.8477198 *     40.15 *
************************************************************


In [12]:
print("Nice!")

Nice!


Now we will calculate the neutrino momenta.

If I understand it correctly:
$$ 80.3^2 = (E_e + E_{\nu})^2 - \lvert \sum \textbf{p} \rvert ^2  $$

$$ 80.3^2 = 80.3^2 - (p_z + p_{z\nu})^2 $$

$$ p_{z\nu}^2 + 2p_zp_{z\nu} + p_z^2 = 0 $$

$$ p_{z\nu} = - p_z $$

So in the stationary W case, electron and neutrino will start moving back to back, so the spatial components will be the same but will have opposite sign?

In that case we can use this to set up another tree with neutrino components:

In [13]:
# Setting up the variables, so we can set up branches, where we store them
nPx = np.array([n4Vect.Px()])
nPy = np.array([n4Vect.Py()])
nPz = np.array([n4Vect.Pz()])
nE = np.array([productE])

In [14]:
# Setting up branches to store components of neutrino momenta 
# (each component is placed separately on each branch)
n.Branch("Px",nPx,"xComponent[1]/D")
n.Branch("Py",nPy,"yComponent[1]/D")
n.Branch("Pz",nPz,"zComponent[1]/D")
n.Branch("E",nE,"energy[1]/D")

In [15]:
# Finding number of entries in the electron tree
e.GetEntries()

3

In [16]:
def neuMomGenerator():
    ''' Generates neutrino momenta components for back to back case
        and fills them into neutrino tree. 
        Same result as elMomGenerator().
    '''
    for entry in e:
        x = -entry.Px
        y = -entry.Py
        z = -entry.Pz
        varE = entry.E

        n4Vect.SetPxPyPzE(x,y,z,varE) #Changing TLorentzVector for neutrino

        nPx[0] = n4Vect.Px()
        nPy[0] = n4Vect.Py()
        nPz[0] = n4Vect.Pz()
        nE[0] = n4Vect.E()

        n.Fill()

In [17]:
# Calling the function
neuMomGenerator()

Now we can look at how the tree looks like.

In [18]:
e.Scan("Px:Py:Pz:E")
n.Scan("Px:Py:Pz:E")

3

************************************************************
*    Row   *        Px *        Py *        Pz *         E *
************************************************************
*        0 * -0.932665 * -0.050281 * -0.357221 *     40.15 *
*        1 * 0.4708793 * -0.563725 * -0.678591 *     40.15 *
*        2 * 0.4350746 * -0.303448 * 0.8477198 *     40.15 *
************************************************************
************************************************************
*    Row   *        Px *        Py *        Pz *         E *
************************************************************
*        0 * 0.9326654 * 0.0502817 * 0.3572210 *     40.15 *
*        1 * -0.470879 * 0.5637255 * 0.6785912 *     40.15 *
*        2 * -0.435074 * 0.3034485 * -0.847719 *     40.15 *
************************************************************


In [19]:
file.Write()
file.Close()

This looks like it could be right and we can check with these values if we decide to read the `stationaryW_case.root` file. Now let's figure out how to read this.

# Reading the File - Mirei

The above is the code I received from Pavel. The only thing I changed above is that I moved the file creating to the beginning. 
Below is the code I have written to read TLorentzVectors from a file. 

Firstly, I just imported the functions. (Pavel imported ROOT as r, but I preferred to write 'root')

In [20]:
import ROOT as root
import numpy as np
from ROOT import TLorentzVector

Then I open the File Pavel created and retrieve the data from the tree called 'e'.
To retrieve the data for the tree 'n', you can just change the 'e' to 'n'.

In [21]:
f=root.TFile.Open("stationaryW_case.root","READ") 
tree=f.Get("e") # where e is the treename


Next, I wasn't sure what the tree 'leaves' were called so I wrote a code to get the leaf names so that I can extract the data from appropriate leaves in order. This is done because I don't think you can just wholly get the data just from the tree. 

In [22]:
leaves = tree.GetListOfLeaves()

# define dynamically a python class containing root Leaves objects
class PyListOfLeaves(dict) :
    pass

# create an istance
pyl = PyListOfLeaves()

And as you can see below, the tree leaf name is outputted

In [23]:
for i in range(0,leaves.GetEntries() ) :
    leaf = leaves.At(i)
    name = leaf.GetName()
    print(name)
    # add dynamically attribute to my class 
    pyl.__setattr__(name,leaf)

xComponent
yComponent
zComponent
energy


Then I extracted the data by looping through each leaf in the TLorentzVector. I put the extracted data into an array.

In [24]:
vector=[]
nev=tree.GetEntries() #number of TLorentzVectors in the tree 'e'
for iev in range(0,nev) :
    tree.GetEntry(iev)
    # get values from the tree using Python class pyl which contains leaves
    # objects 
    px = pyl.xComponent.GetValue()
    py = pyl.yComponent.GetValue()
    pz = pyl.zComponent.GetValue()
    Energy=pyl.energy.GetValue()
    
    vector.append((px,py,pz,Energy))
    if iev == 10:
        break
        
print(vector)

[(-0.9326654560616809, -0.05028174874714363, -0.35722106994434943, 40.15), (0.4708793779444828, -0.563725536037116, -0.678591284534574, 40.15), (0.43507467855000875, -0.30344852135453076, 0.8477198941704479, 40.15)]
